** Final project code **

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import pygsp as pg

In [ ]:
from load_and_preprocessing import load_data_and_filter_members, assign_party_to_names
from visualization import label_to_numeric, get_lap_eigendecomp, detect_partitions, make_signal

In [ ]:
# Use load_data_and_filter_members to create adjacency for any file from any legislature
# Example 
leg='50'
from_date = '01_01_2018'
to_date = '31_12_2018'
adjacency, node_index, sum_na_per_row = load_data_and_filter_members('../data/abdb-de-all-affairs-'+leg+'-0.csv',
                                                                     start_date=from_date, end_date=to_date,
                                                                     filter_method='number_NA',cutoff=10,ret_transf=False)

Assigning parties to the councillors from the dataset loaded above:

In [ ]:
name_with_party = assign_party_to_names('../data/Ratsmitglieder_1848_FR.csv', node_index)
print(name_with_party.to_string())

Make a csv that can be used in Gephi:

In [ ]:
name_labels = name_with_party['CouncillorName'].values
adjacency_df = pd.DataFrame(data=adjacency, columns=name_labels, index=name_labels)
adjacency_df.to_csv('gephi_'+leg+'-'+from_date+'-'+to_date+'.csv', sep=',', index_label='', index=True, header=True)

Translating the party label into a numerical value:

In [ ]:
# Example here is with parties, but the same function works for any kind of dictionary, eg. lobbying mandates
party_map = {'UDC': 6,'PSS': -6,'PDC':0,'BastA':-3,'PLR':5,'pvl':-2,
             'PES':-5, 'PBD':2, 'PdT':-4,'PLS':5,
             'PRD':5, 'MCR':4, 'PEV':-1, 'Lega':3, 'csp-ow':1, 'Al':-7,'FraP!':-8, 'GB':7, 'CSPO':8 }

name_with_party_num, labels_in_data = label_to_numeric(name_with_party, 'PartyAbbreviation', party_map, ret_values=True)

This can be used to see if the label dictionary needs to be adjusted:

In [ ]:
print("Unique label values found in data: \n{0}".format(labels_in_data))
print("Label values that are translated by provided dictionary: \n{0}".format(party_map.keys()))

Find partitions based on Louvain method and calculate modularity

In [ ]:
partitions, modularity = detect_partitions(adjacency, resolution=0.5)

Draw some eigenmaps and plot party membership as signal:

In [ ]:
eigenvals, eigenvectors = get_lap_eigendecomp(adjacency, lap_type='normalized', ret_eigval=True)

In [ ]:
%pylab

In [ ]:
partition_signal = make_signal(adjacency.shape[0],partitions)
party_signal = name_with_party_num['PartyAbbreviation'].values
colorbar_labels = ['UDC', 'PLR', 'MCR', 'LEGA', 'PBD', 'csp-ow', 'PDC', 'PEV', 'pvl', 'BastA', 'PdT', 'PES', 'PSS']

In [ ]:
fig = plt.figure(figsize=(16,9))
ax = fig.add_subplot(111)
ax.scatter(eigenvectors[:,1], eigenvectors[:,2], c=partition_signal, s=15, vmin=-6, vmax=6, cmap="jet")

for i, txt in enumerate(name_labels):
    ax.annotate(txt, (eigenvectors[i,1], eigenvectors[i,2]), xytext=(1,1), textcoords='offset points')
    
ax.set_title('Position on political spectrum by party ')
fig.tight_layout()

Set colorbar to whatever

In [ ]:
cbar = fig.colorbar(ax.collections[0], ticks=np.linspace(6,-6,13), orientation='vertical')
cbar.ax.set_yticklabels(colorbar_labels)